# Importing Packages

In [2]:
import requests  # Import the requests library for making HTTP requests
import pandas as pd  # Import the pandas library for data manipulation
import re  # Import the re module for regular expressions
import numpy as np  # Import the numpy library for numerical operations
import json  # Import the json module for handling JSON data
import os
import scipy.stats as stats
import configparser

config = configparser.ConfigParser()
config.read("my_config.ini")

pd.options.display.max_rows = None  # Display all rows of a DataFrame
pd.options.display.max_columns = None  # Display all columns of a DataFrame
pd.options.mode.chained_assignment = None

In [2]:
api_key = config["census"]["api_key"]

# Download from API

In [ ]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

# Define a list of group codes for which data will be downloaded
group_code_list = ["DP02", "DP03", "DP04", "DP05"]


# Define a function to download data via the Census Bureau API
def download_dp_via_api(year_list, group_code_list, KEY):
    # Create an empty list to store DataFrames for each year
    year_df_list = []

    # Loop through each year in the year_list
    for YEAR in year_list:
        # Create an empty list to store DataFrames for each group code
        group_code_df_list = []
        print("Year: ", YEAR)

        # Loop through each group code in the group_code_list
        for VARIABLE in group_code_list:

            print("Group Code: ", VARIABLE)

            # Define the API endpoint URL for the specific year, group code, and geographic area
            url = f"https://api.census.gov/data/{YEAR}/acs/acs5/profile?get=group({VARIABLE})&for=county:*&key={KEY}"
            try:
                # Send a GET request to the API endpoint
                response = requests.get(url)
                # Check if the request was successful (status code 200)
                if response.status_code == 200:
                    # Parse the JSON response
                    data = response.json()
                    # Extract column names (headers) and data rows
                    columns = data[0]
                    df_data = [data[i] for i in range(1, len(data) - 1)]
                    # Create a DataFrame from the extracted data
                    df = pd.DataFrame(df_data, columns=columns)
                else:
                    # Print an error message if the request was not successful
                    print("Request failed with status code:", response.status_code)
            except requests.exceptions.RequestException as e:
                # Handle exceptions related to the request (e.g., network issues)
                print("Request error:", e)

            if YEAR not in ["2020", "2021", "2022"]:
                df["CODE"] = df.state + df.county
            else:
                df["CODE"] = df["GEO_ID"].astype(str)[-5:]

            # Extract only the estimate columns (ending in 'E')
            headings_list = []

            # Find all column headers that match regex expression of E at the end of the string
            for heading in df.columns:
                if re.match(r".*\dE$", heading):
                    headings_list.append(heading)

            # Add back the column headings GEO_ID, NAME, and CODE to the DataFrame
            headings_list.extend(["GEO_ID", "NAME", "CODE"])

            # Take a subset of the original dataframe for headers ending in E only
            df = df[headings_list]

            # Retrieve JSON for label names for header codes
            url = f"https://api.census.gov/data/{YEAR}/acs/acs5/profile/variables.json"
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    data_2 = response.json()
                    # Extract variable information
                    variable_dict = data_2["variables"]
                else:
                    # Print an error message if the request was not successful
                    print("Request failed with status code:", response.status_code)
            except requests.exceptions.RequestException as e:
                # Handle exceptions related to the request (e.g., network issues)
                print("Request error:", e)

            # Create a list of column names based on variable labels
            column_names = [variable_dict[code]["label"] for code in df.columns[:-3]]

            column_names_processed = [
                str(label.replace("!!", " "))[9:] for label in column_names
            ]

            # Add back the column headings GEO_ID, NAME, and CODE to the DataFrame
            column_names_processed.extend(["GEO_ID", "NAME", "CODE"])
            df.loc[len(df.index)] = df.columns
            df.columns = column_names_processed

            # Append the DataFrame for the current group code to the list
            group_code_df_list.append(df)

        # Merge DataFrames for all group codes for the current year
        year_df = (
            group_code_df_list[0]
            .merge(group_code_df_list[1], on=["GEO_ID", "NAME", "CODE"])
            .merge(group_code_df_list[2], on=["GEO_ID", "NAME", "CODE"])
            .merge(group_code_df_list[3], on=["GEO_ID", "NAME", "CODE"])
        )

        year_df.to_csv(f"data/census/df_{YEAR}.csv")

        variable_name = f"df_{YEAR}"
        globals()[variable_name] = year_df

        # Append the merged DataFrame for the current year to the list
        year_df_list.append(year_df)

    return year_df_list


# Call the download_via_api function with the specified parameters
year_df_list = download_dp_via_api(year_list, group_code_list, api_key)

# Clean Headers

In [3]:
year_list = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]

# Define a function named 'clean_headers' that takes dataframe as a parameter
def clean_headers(year):
    # Read a CSV file into a pandas DataFrame based on the provided 'year'
    df = pd.read_csv(f"data/census/df_{year}.csv", index_col=0, low_memory=False)

    # Initialize an empty list to store new column names
    new_header_list = []

    # Open and read a JSON file named "conversion.json"
    f = open("json/header_conversion.json")
    conversion = json.load(f)

    # Initialize an empty list to store header names that couldn't be converted
    error_list = []

    df.columns = [str(x).replace('"', "") for x in df.columns.tolist()]

    # Iterate through the columns of the DataFrame and attempt to convert the headers
    for header in df.columns.tolist():
        try:
            # Try to find a new header in the "headers" dictionary within "conversion"
            new_header = conversion["headers"][header]
            new_header_list.append(new_header)
        except KeyError as e:
            # If the header is not found in "conversion," add it to the error list
            print(str(e))
            error_list.append(str(e))
            new_header_list.append(header)

    # Assign the modified 'new_header_list' as the new column names of the DataFrame
    df.columns = new_header_list

    # Add a new column 'year' with the provided 'year' value to the DataFrame
    df["year"] = int(year)

    df = df.loc[:, ~df.columns.duplicated()]

    # Return the modified DataFrame
    return df


# Call the 'clean_headers' function for years 2012, 2016, and 2020, and store the results in separate DataFrames
df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022 = [clean_headers(year) for year in year_list]

# Further Data Cleaning

In [4]:
# Define a function named 'clean_up' that takes a DataFrame 'df' as a parameter
def clean_up(df):
    # Replace specific numeric values with NaN (Not-a-Number) in the DataFrame
    df = df.replace("-888888888", np.nan)
    df = df.replace("-999999999", np.nan)
    df = df.replace("-888888888.0", np.nan)
    df = df.replace("-999999999,0", np.nan)

    # Drop columns with more than 20% missing data (NaN)
    df = df.dropna(thresh=0.50 * len(df), axis=1)

    # Drop rows where the "GEO_ID" column equals "GEO_ID"
    df = df.drop(df.loc[df.GEO_ID == "GEO_ID"].index)

    # Return the cleaned DataFrame
    return df


# Apply the 'clean_up' function to each of the DataFrames for years 2012, 2016, and 2020
(
    df_2010,
    df_2011,
    df_2012,
    df_2013,
    df_2014,
    df_2015,
    df_2016,
    df_2017,
    df_2018,
    df_2019,
    df_2020,
    df_2021,
    df_2022,
) = [
    clean_up(df)
    for df in (
        df_2010,
        df_2011,
        df_2012,
        df_2013,
        df_2014,
        df_2015,
        df_2016,
        df_2017,
        df_2018,
        df_2019,
        df_2020,
        df_2021,
        df_2022,
    )
]


# Define a function named 'clean_geo' that takes a DataFrame 'df' as a parameter
def clean_geo(df):
    # # Create a new column 'geocode' based on the last 5 or 2 characters of the 'GEO_ID' column
    df["geo_code"] = df["GEO_ID"].apply(
        lambda x: str(x)[-5:] if len(str(x)) > 11 else str(x)[-2:]
    )

    # Split the 'NAME' column into a 'state' column when a comma is present and keep the second part
    df["state"] = np.where(
        df["NAME"].str.contains(pat=",", regex=True),
        df["NAME"].str.split(","),
        df["NAME"],
    )

    # Apply a lambda function to 'state' column to get the second part if it's a list
    df["state"] = df["state"].apply(lambda x: x[1] if type(x) == list else x)
    df["state"] = df["state"].apply(lambda x: str(x).strip())

    with open("json/state_conversion.json", "r") as fp:
        state_conversion = json.load(fp)

    df["state_code"] = df.state.apply(lambda x: state_conversion[x])

    df["geo_full_name"] = df.NAME.copy()

    # Create a new column 'geoname' based on the first part of the 'NAME' column when a comma is present
    df["geo_name"] = np.where(
        df["NAME"].str.contains(pat=",", regex=True),
        df["NAME"].str.split(","),
        df["NAME"],
    )
    df["geo_name"] = df["geo_name"].apply(lambda x: x[0] if type(x) == list else x)

    df["geo_name"] = df["geo_name"].str.replace(" County", "")

    # Drop the 'GEO_ID' column
    df = df.drop(columns=["NAME"])

    # Return the cleaned DataFrame
    return df


# Apply the 'clean_geo' function to each of the DataFrames for years 2012, 2016, and 2020
(
    df_2010,
    df_2011,
    df_2012,
    df_2013,
    df_2014,
    df_2015,
    df_2016,
    df_2017,
    df_2018,
    df_2019,
    df_2020,
    df_2021,
    df_2022,
) = [
    clean_geo(df)
    for df in (
        df_2010,
        df_2011,
        df_2012,
        df_2013,
        df_2014,
        df_2015,
        df_2016,
        df_2017,
        df_2018,
        df_2019,
        df_2020,
        df_2021,
        df_2022,
    )
]

In [5]:
census = pd.concat(
    [
        df_2010,
        df_2011,
        df_2012,
        df_2013,
        df_2014,
        df_2015,
        df_2016,
        df_2017,
        df_2018,
        df_2019,
        df_2020,
        df_2021,
        df_2022
    ], axis=0
)

census.head()

,speak_non_english,speak_non_english_speak_english_not_well,speak_spanish,speak_spanish_speak_english_not_well,speak_other_indo,speak_other_indo_speak_english_not_well,speak_asian_pacific,speak_asian_pacific_speak_english_not_well,speak_other,speak_other_speak_english_not_well,ancestry_total_pop,ancestry_american,ancestry_arab,ancestry_czech,ancestry_danish,ancestry_dutch,ancestry_english,ancestry_french,ancestry_french_canadian,ancestry_german,ancestry_greek,ancestry_hungarian,ancestry_irish,ancestry_italian,ancestry_lithuanian,ancestry_norwegian,ancestry_polish,ancestry_portuguese,ancestry_russian,ancestry_scotch_irish,ancestry_scottish,ancestry_slovak,ancestry_subsaharan_african,ancestry_swedish,ancestry_swiss,ancestry_ukrainian,ancestry_welsh,ancestry_west_indian,total_hh,family_hh,family_hh_own_children,married_couple_hh,married_couple_hh_own_children,sole_male_hh,sole_male_hh_own_children,sole_female_hh,sole_female_hh_own_children,nonfamily_hh,nonfamily_hh_living_alone,nonfamily_hh_living_alone_65_plus,hh_with_1_plus_under_18,hh_with_1_plus_65_plus,average_hh_size,average_family_size,pop_in_hh,householder,spouse,child,other_relatives,nonrelatives,unmarried_partner,marital_status_males_15_plus,males_never_married,males_married,males_separated,males_widowed,males_divorced,marital_status_females_15_plus,females_never_married,females_married,females_separated,females_widowed,females_divorced,women_15_50_yo_with_birth,unmarried_women,unmarried_women_per_1000,per_1000_unmarried_women_15_50_yo,per_1000_15_50_yo_women_15_19_yo,per_1000_15_50_yo_women_20_34_yo,per_1000_15_50_yo_women_35_50_yo,grandparents_with_grandkids,responsible_for_grandkids,yrs_responsible_less_than_1_yr,yrs_responsible_1_to_2_yrs,yrs_responsible_3_to_4_yrs,yrs_responsible_5_plus_yrs,grandparents_resp_for_grandkids,female_grandparents,married_grandparents,pop_3_plus_enrolled,nursery_pre_school,kindergarten,elementary_1_8,high_school_9_12,college_or_graduate_school,pop_25_plus_educ_attainment,less_than_9th_grade,9th_12th_grade_no_diploma,high_school_graduate,some_college_no_degree,associates_degree,bachelors_degree,graduate_or_professional_degree,civilian_pop_18_plus,civilian_veterans,residence_year_ago_pop_1_plus,same_house,diff_house_in_us,diff_house_in_us_same_county,diff_house_in_us_diff_county,diff_house_in_us_diff_county_same_state,diff_house_in_us_diff_county_diff_state,diff_house_abroad,place_of_birth_total_pop,native,native_born_in_us,native_born_in_us_state_of_residence,native_born_in_us_diff_state,native_born_in_pr_us_islands_abroad,foreign_born,foreign_born_pop_citizenship,naturalized_citizen,not_us_citizen,year_entry_pop_born_outside_us,year_entry_native,native_entered_2000_or_later,native_entered_before_2000,year_entry_foreign_born,foreign_entered_2000_or_later,foreign_entered_before_2000,foreign_born_pop,born_europe,born_asia,born_africa,born_oceania,born_latin_america,born_northern_america,pop_5_plus_spoken_at_home,speak_english_only,GEO_ID,CODE,employment_status_pop_16_plus,in_labor_force,in_civilian_lf,in_civilian_lf_employed,in_civilian_lf_unemployed,in_armed_forces,not_in_labor_force,civilian_lf,employment_status_females_16_plus,females_in_lf,females_in_civilian_lf,females_in_civilian_lf_employed,own_children_under_6,all_parents_in_family_in_lf_under_6,own_children_6_17,all_parents_in_family_in_lf_6_17,commuters_16_plus,drove_alone,carpooled,public_transportation,walked,other_commuting_means,worked_at_home,mean_travel_time,civilian_employed_pop_16_plus,management_business_science_arts_occupations,service_occupations,sales_and_office_occupations,natural_resources_construction_maintenance_occupations,production_transportation_material_moving_occupations,industry_civilian_employed_pop_16_plus,agriculture_hunting_and_mining,construction,manufacturing,wholesale_trade,retail_trade,transportation_and_utilities,information,finance_insurance_and_real_estate,professional_management_and_scientific_services,educational_and_health_services,arts_enterta

In [58]:
len(census)

41855

In [59]:
census.year.value_counts()

year
2022    3221
2010    3220
2011    3220
2012    3220
2013    3220
2020    3220
2021    3220
2014    3219
2015    3219
2016    3219
2017    3219
2018    3219
2019    3219
Name: count, dtype: int64

# Feature Selection and Engineering

In [60]:
census.head()

,speak_non_english,speak_non_english_speak_english_not_well,speak_spanish,speak_spanish_speak_english_not_well,speak_other_indo,speak_other_indo_speak_english_not_well,speak_asian_pacific,speak_asian_pacific_speak_english_not_well,speak_other,speak_other_speak_english_not_well,ancestry_total_pop,ancestry_american,ancestry_arab,ancestry_czech,ancestry_danish,ancestry_dutch,ancestry_english,ancestry_french,ancestry_french_canadian,ancestry_german,ancestry_greek,ancestry_hungarian,ancestry_irish,ancestry_italian,ancestry_lithuanian,ancestry_norwegian,ancestry_polish,ancestry_portuguese,ancestry_russian,ancestry_scotch_irish,ancestry_scottish,ancestry_slovak,ancestry_subsaharan_african,ancestry_swedish,ancestry_swiss,ancestry_ukrainian,ancestry_welsh,ancestry_west_indian,total_hh,family_hh,family_hh_own_children,married_couple_hh,married_couple_hh_own_children,sole_male_hh,sole_male_hh_own_children,sole_female_hh,sole_female_hh_own_children,nonfamily_hh,nonfamily_hh_living_alone,nonfamily_hh_living_alone_65_plus,hh_with_1_plus_under_18,hh_with_1_plus_65_plus,average_hh_size,average_family_size,pop_in_hh,householder,spouse,child,other_relatives,nonrelatives,unmarried_partner,marital_status_males_15_plus,males_never_married,males_married,males_separated,males_widowed,males_divorced,marital_status_females_15_plus,females_never_married,females_married,females_separated,females_widowed,females_divorced,women_15_50_yo_with_birth,unmarried_women,unmarried_women_per_1000,per_1000_unmarried_women_15_50_yo,per_1000_15_50_yo_women_15_19_yo,per_1000_15_50_yo_women_20_34_yo,per_1000_15_50_yo_women_35_50_yo,grandparents_with_grandkids,responsible_for_grandkids,yrs_responsible_less_than_1_yr,yrs_responsible_1_to_2_yrs,yrs_responsible_3_to_4_yrs,yrs_responsible_5_plus_yrs,grandparents_resp_for_grandkids,female_grandparents,married_grandparents,pop_3_plus_enrolled,nursery_pre_school,kindergarten,elementary_1_8,high_school_9_12,college_or_graduate_school,pop_25_plus_educ_attainment,less_than_9th_grade,9th_12th_grade_no_diploma,high_school_graduate,some_college_no_degree,associates_degree,bachelors_degree,graduate_or_professional_degree,civilian_pop_18_plus,civilian_veterans,residence_year_ago_pop_1_plus,same_house,diff_house_in_us,diff_house_in_us_same_county,diff_house_in_us_diff_county,diff_house_in_us_diff_county_same_state,diff_house_in_us_diff_county_diff_state,diff_house_abroad,place_of_birth_total_pop,native,native_born_in_us,native_born_in_us_state_of_residence,native_born_in_us_diff_state,native_born_in_pr_us_islands_abroad,foreign_born,foreign_born_pop_citizenship,naturalized_citizen,not_us_citizen,year_entry_pop_born_outside_us,year_entry_native,native_entered_2000_or_later,native_entered_before_2000,year_entry_foreign_born,foreign_entered_2000_or_later,foreign_entered_before_2000,foreign_born_pop,born_europe,born_asia,born_africa,born_oceania,born_latin_america,born_northern_america,pop_5_plus_spoken_at_home,speak_english_only,GEO_ID,CODE,employment_status_pop_16_plus,in_labor_force,in_civilian_lf,in_civilian_lf_employed,in_civilian_lf_unemployed,in_armed_forces,not_in_labor_force,civilian_lf,employment_status_females_16_plus,females_in_lf,females_in_civilian_lf,females_in_civilian_lf_employed,own_children_under_6,all_parents_in_family_in_lf_under_6,own_children_6_17,all_parents_in_family_in_lf_6_17,commuters_16_plus,drove_alone,carpooled,public_transportation,walked,other_commuting_means,worked_at_home,mean_travel_time,civilian_employed_pop_16_plus,management_business_science_arts_occupations,service_occupations,sales_and_office_occupations,natural_resources_construction_maintenance_occupations,production_transportation_material_moving_occupations,industry_civilian_employed_pop_16_plus,agriculture_hunting_and_mining,construction,manufacturing,wholesale_trade,retail_trade,transportation_and_utilities,information,finance_insurance_and_real_estate,professional_management_and_scientific_services,educational_and_health_services,arts_enterta

## Not Proficient In English

% Not Proficient in English: % of population aged 5 and over who reported speaking English less than "well"

In [6]:
english = census[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "pop_5_plus_spoken_at_home",
        "speak_non_english_speak_english_not_well",
        "speak_spanish_speak_english_not_well",
        "speak_other_indo_speak_english_not_well",
        "speak_asian_pacific_speak_english_not_well",
        "speak_other_speak_english_not_well",
    ]
]

In [7]:
english["pop_5_plus_spoken_at_home"] = english["pop_5_plus_spoken_at_home"].astype(
    float
)
english["speak_non_english_speak_english_not_well"] = english[
    "speak_non_english_speak_english_not_well"
].astype(float)

english["pct_not_proficient_in_english"] = (
    english["speak_non_english_speak_english_not_well"]
    / english["pop_5_plus_spoken_at_home"]
)

eng = english[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "pct_not_proficient_in_english",
    ]
]

eng.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574


## Uninsured, Uninsured Adults, Uninsured Children

- Uninsured: % of population under age 65 without health insurances
- Uninsured Adults: % of adults under 65 without health insurance
- Uninsured Children:  % of children under age 19 without health insurance

In [8]:
healthcare = census[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "health_insurance_civilian_noninst_pop",
        "no_hi_coverage",
        "hi_civilian_noninst_pop_under_18",
        "no_hi_coverage_under_18",
        "hi_civilian_noninst_population_18_64",
        "hi_in_lf_employed_no_hi_coverage",
        "hi_in_lf_unemployed_no_hi_coverage",
        "hi_not_in_lf_no_hi_coverage",
    ]
]

healthcare["health_insurance_civilian_noninst_pop"] = healthcare[
    "health_insurance_civilian_noninst_pop"
].astype(float)
healthcare["no_hi_coverage"] = healthcare["no_hi_coverage"].astype(float)
healthcare["hi_civilian_noninst_pop_under_18"] = healthcare[
    "hi_civilian_noninst_pop_under_18"
].astype(float)
healthcare["no_hi_coverage_under_18"] = healthcare["no_hi_coverage_under_18"].astype(
    float
)
healthcare["hi_civilian_noninst_population_18_64"] = healthcare[
    "hi_civilian_noninst_population_18_64"
].astype(float)
healthcare["hi_in_lf_employed_no_hi_coverage"] = healthcare[
    "hi_in_lf_employed_no_hi_coverage"
].astype(float)
healthcare["no_hi_lf_unemployed_no_hi_coverage"] = healthcare[
    "hi_in_lf_unemployed_no_hi_coverage"
].astype(float)
healthcare["hi_not_in_lf_no_hi_coverage"] = healthcare[
    "hi_not_in_lf_no_hi_coverage"
].astype(float)
healthcare["hi_civilian_noninst_population_18_64"] = healthcare[
    "hi_civilian_noninst_population_18_64"
].astype(float)

healthcare["uninsured"] = (
    healthcare["no_hi_coverage"] / healthcare["health_insurance_civilian_noninst_pop"]
)
healthcare["uninsured_children"] = (
    healthcare["no_hi_coverage_under_18"]
    / healthcare["hi_civilian_noninst_pop_under_18"]
)
healthcare["uninsured_adults"] = (
    healthcare["hi_in_lf_employed_no_hi_coverage"].astype(float)
    + healthcare["hi_in_lf_unemployed_no_hi_coverage"].astype(float)
    + healthcare["hi_not_in_lf_no_hi_coverage"].astype(float)
) / healthcare["hi_civilian_noninst_population_18_64"].astype(float)

health = healthcare[
    ["geo_code", "year", "uninsured", "uninsured_children", "uninsured_adults"]
]

merge = eng.merge(health, on=["geo_code", "year"], how='left')
merge.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN


In [98]:
merge.loc[merge.uninsured.notna()].head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults
6440,2012,29115,Missouri,MO,"Linn County, Missouri",Linn,0.011533,0.113241,0.091707,0.159439
6441,2012,29091,Missouri,MO,"Howell County, Missouri",Howell,0.014389,0.165452,0.096062,0.240947
6442,2012,29101,Missouri,MO,"Johnson County, Missouri",Johnson,0.009686,0.117842,0.063999,0.156827
6443,2012,29105,Missouri,MO,"Laclede County, Missouri",Laclede,0.005514,0.171635,0.091885,0.248406
6444,2012,29125,Missouri,MO,"Maries County, Missouri",Maries,0.007811,0.122356,0.046773,0.189766


## Long Commute Driving Alone, Driving Alone to Work

- Long Commute Driving Alone: Among workers who commute in their car alone, % that commute more than 30 minutes
- Driving Alone to Work:  % of workforce that drives alone to work

In [9]:
commute = census[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "commuters_16_plus",
        "drove_alone",
        "mean_travel_time"
    ]
]

commute["drove_alone"] = commute["drove_alone"].astype(float)
commute["mean_travel_time"] = commute["mean_travel_time"].astype(float)
commute["commuters_16_plus"] = commute["commuters_16_plus"].astype(float)


# Define a function to calculate the percentage of commuters driving alone for more than 30 minutes
def calculate_percentage(row):
    threshold_time = 30  # Threshold time in minutes
    std_dev_travel_time = (
        10  # Estimated standard deviation (can be adjusted based on data)
    )

    # Step 1: Calculate z-score
    z_score = (threshold_time - row["mean_travel_time"]) / std_dev_travel_time

    # Step 2: Calculate the proportion of commuters with travel time > 30 minutes
    proportion_above_30 = 1 - stats.norm.cdf(z_score)

    # Step 3: Calculate the number of commuters driving alone for more than 30 minutes
    commuters_above_30 = proportion_above_30 * row["drove_alone"]

    # Step 4: Calculate the percentage of commuters driving alone for more than 30 minutes
    percentage_commuters_above_30 = (commuters_above_30 / row["drove_alone"]) * 100

    return percentage_commuters_above_30


# Apply the function to each row and create a new column for the result
commute["long_commute_driving_alone"] = commute.apply(calculate_percentage, axis=1)

commute["driving_alone_to_work"] = commute["drove_alone"] / commute["commuters_16_plus"]

com = commute[
    ["geo_code", "year", "long_commute_driving_alone", "driving_alone_to_work"]
]

merge_2 = merge.merge(com, on=["geo_code", "year"], how="left")
merge_2.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110


## Income Inequality

- Income Inequality: Ratio of household income at the 80th percentile to income at the 20th percentile

In [10]:
income = census[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "income_total_hh",
        "hh_income_10000_less",
        "hh_income_10000_14999",
        "hh_income_15000_24999",
        "hh_income_25000_34999",
        "hh_income_35000_49999",
        "hh_income_50000_74999",
        "hh_income_75000_99999",
        "hh_income_100000_149999",
        "hh_income_150000_199999",
        "hh_income_200000_plus",
    ]
]

income["hh_income_0_14999"] = income["hh_income_10000_less"].astype(float) + income[
    "hh_income_10000_14999"
].astype(float)
income["hh_income_0_24999"] = income["hh_income_0_14999"].astype(float) + income[
    "hh_income_15000_24999"
].astype(float)
income["hh_income_0_34999"] = income["hh_income_0_24999"].astype(float) + income[
    "hh_income_25000_34999"
].astype(float)
income["hh_income_0_49999"] = income["hh_income_0_34999"].astype(float) + income[
    "hh_income_35000_49999"
].astype(float)
income["hh_income_0_74999"] = income["hh_income_0_49999"].astype(float) + income[
    "hh_income_50000_74999"
].astype(float)
income["hh_income_0_99999"] = income["hh_income_0_74999"].astype(float) + income[
    "hh_income_75000_99999"
].astype(float)
income["hh_income_0_149999"] = income["hh_income_0_99999"].astype(float) + income[
    "hh_income_100000_149999"
].astype(float)
income["hh_income_0_199999"] = income["hh_income_0_149999"].astype(float) + income[
    "hh_income_150000_199999"
].astype(float)
income["hh_income_0_200000"] = income["hh_income_0_199999"].astype(float) + income[
    "hh_income_200000_plus"
].astype(float)

income["p_20"] = income["income_total_hh"].astype(float) * 0.2
income["p_80"] = income["income_total_hh"].astype(float) * 0.8


geo = income[["year", "geo_code", "state", "state_code", "geo_full_name", "geo_name"]]


income["hh_income_10000_less"] = income["hh_income_10000_less"].astype(float)
income["hh_income_0_14999"] = income["hh_income_0_14999"].astype(float)
income["hh_income_0_24999"] = income["hh_income_0_24999"].astype(float)
income["hh_income_0_34999"] = income["hh_income_0_34999"].astype(float)
income["hh_income_0_49999"] = income["hh_income_0_49999"].astype(float)
income["hh_income_0_74999"] = income["hh_income_0_74999"].astype(float)
income["hh_income_0_99999"] = income["hh_income_0_99999"].astype(float)
income["hh_income_0_149999"] = income["hh_income_0_149999"].astype(float)
income["hh_income_0_199999"] = income["hh_income_0_199999"].astype(float)

inc = income[["hh_income_10000_less",
                "hh_income_0_14999",
                "hh_income_0_24999",
                "hh_income_0_34999",
                "hh_income_0_49999",
                "hh_income_0_74999",
                "hh_income_0_99999",
                "hh_income_0_149999",
                "hh_income_0_199999",
                "p_20",
                "p_80"]]
inc.head()

,hh_income_10000_less,hh_income_0_14999,hh_income_0_24999,hh_income_0_34999,hh_income_0_49999,hh_income_0_74999,hh_income_0_99999,hh_income_0_149999,hh_income_0_199999,p_20,p_80
0,491.0,1066.0,1833.0,2679.0,3662.0,5088.0,5787.0,6170.0,6263.0,1258.6,5034.4
1,235.0,540.0,970.0,1520.0,2263.0,3374.0,4090.0,4847.0,4979.0,1011.2,4044.8
2,368.0,690.0,1323.0,2093.0,3395.0,4799.0,5919.0,6549.0,6723.0,1372.4,5489.6
3,277.0,730.0,1803.0,2576.0,4252.0,6775.0,8748.0,11007.0,11898.0,2478.2,9912.8
4,876.0,1621.0,3524.0,5083.0,6812.0,8923.0,10414.0,11338.0,11525.0,2339.6,9358.4


In [11]:
# Dictionary for bracket lengths and lower bounds
bracket_lengths = {
    "hh_income_10000_less": 10000,
    "hh_income_0_14999": 5000,
    "hh_income_0_24999": 10000,
    "hh_income_0_34999": 10000,
    "hh_income_0_49999": 15000,
    "hh_income_0_74999": 25000,
    "hh_income_0_99999": 25000,
    "hh_income_0_149999": 50000,
    "hh_income_0_199999": 50000
}
lower_bounds = {
    "hh_income_10000_less": 0,
    "hh_income_0_14999": 10000,
    "hh_income_0_24999": 15000,
    "hh_income_0_34999": 25000,
    "hh_income_0_49999": 35000,
    "hh_income_0_74999": 50000,
    "hh_income_0_99999": 75000,
    "hh_income_0_149999": 100000,
    "hh_income_0_199999": 150000
}


# Function to perform the calculation
def calculate_20_value(row):
    # Iterate through each column in the correct order
    for i, col in enumerate(bracket_lengths.keys()):
        if row["p_20"] < row[col]:
            # Find the previous column
            prev_col = list(bracket_lengths.keys())[i - 1] if i > 0 else None

            # Ensure there's a previous column for comparison
            if prev_col:
                # Calculate the numerator and denominator
                numerator = row["p_20"] - row[prev_col]
                denominator = row[col] - row[prev_col]

                # Fraction calculation
                fraction = numerator / denominator

                # Multiply by the bracket length
                product = fraction * bracket_lengths[col]

                # Add to the lower bound of the bracket
                result = lower_bounds[col] + product
                return result
    return None  # If p_20 is greater than all columns


# Apply the function to each row and store the result in a new column
inc["hh_income_20"] = inc.apply(calculate_20_value, axis=1)


# Function to perform the calculation
def calculate_80_value(row):
    # Iterate through each column in the correct order
    for i, col in enumerate(bracket_lengths.keys()):
        if row["p_80"] < row[col]:
            # Find the previous column
            prev_col = list(bracket_lengths.keys())[i - 1] if i > 0 else None

            # Ensure there's a previous column for comparison
            if prev_col:
                # Calculate the numerator and denominator
                numerator = row["p_80"] - row[prev_col]
                denominator = row[col] - row[prev_col]

                # Fraction calculation
                fraction = numerator / denominator

                # Multiply by the bracket length
                product = fraction * bracket_lengths[col]

                # Add to the lower bound of the bracket
                result = lower_bounds[col] + product
                return result
    return None  # If p_80 is greater than all columns


# Apply the function to each row and store the result in a new column
inc["hh_income_80"] = inc.apply(calculate_80_value, axis=1)

inc["income_inequality"] = inc["hh_income_80"] / inc["hh_income_20"]

income_df = pd.concat([geo, inc], axis=1)

inc_df = income_df[["year", "geo_code", "income_inequality"]]

merge_3 = merge_2.merge(inc_df, on=["geo_code", "year"], how='left')
merge_3.head()

/var/folders/73/xg_s806n4y53k4bc7l5f6l800000gn/T/ipykernel_37348/3220697275.py:41: RuntimeWarning: divide by zero encountered in scalar divide
  fraction = numerator / denominator


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247


## Age Distribution

- Population: Resident population
- % below 18 years of age: % population below 18 years of age
- % 65 and older: % population ages 65 and older
- Male population 0-17
- Male population 18-44
- Male population 45-64
- Male population 65+
- Total male population
- Female population 0-17
- Female population 18-44
- Female population 45-64
- Female population 65+ 
- Total female population
- % Females
- Population growth 

In [12]:
age_gender = census[
    [
        "year",
        "geo_code",
        "state",
        "state_code",
        "geo_full_name",
        "geo_name",
        "sex_age_total_pop",
        "male_pop",
        "female_pop",
        "pop_5_below",
        "pop_5_9",
        "pop_10_14",
        "pop_15_19",
        "pop_20_24",
        "pop_25_34",
        "pop_35_44",
        "pop_45_54",
        "pop_55_59",
        "pop_60_64",
        "pop_65_74",
        "pop_75_84",
        "pop_85_plus",
        "median_age",
        "pop_18_plus",
        "pop_21_plus",
        "pop_62_plus",
        "pop_65_plus",
    ]
]

age_gender["population"] = age_gender["sex_age_total_pop"].astype(int)
age_gender["under_18"] = age_gender["sex_age_total_pop"].astype(int) - age_gender[
    "pop_18_plus"
].astype(int)
age_gender["age_45_64"] = (
    age_gender["pop_45_54"].astype(int)
    + age_gender["pop_55_59"].astype(int)
    + age_gender["pop_60_64"].astype(int)
)
age_gender["age_18_44"] = (
    age_gender["sex_age_total_pop"].astype(int)
    - age_gender["under_18"].astype(int)
    - age_gender["pop_65_plus"].astype(int)
    - age_gender["age_45_64"].astype(int)
)
age_gender["pct_under_18"] = age_gender["under_18"].astype(int) / age_gender[
    "sex_age_total_pop"
].astype(int)
age_gender["pct_age_18_44"] = age_gender["age_18_44"].astype(int) / age_gender[
    "sex_age_total_pop"
].astype(int)
age_gender["pct_age_45_64"] = age_gender["age_45_64"].astype(int) / age_gender[
    "sex_age_total_pop"
].astype(int)
age_gender["pct_65_and_older"] = age_gender["pop_65_plus"].astype(int) / age_gender[
    "sex_age_total_pop"
].astype(int)

age_gender = age_gender[
    [
        "year",
        "geo_code",
        "population",
        "median_age",
        "pct_under_18",
        "pct_age_18_44",
        "pct_age_45_64",
        "pct_65_and_older",
    ]
]

merge_4 = merge_3.merge(age_gender, on=["geo_code", "year"], how='left')
merge_4.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485


## Sex by Age

In [15]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

sex_age_list = []

for year in year_list:
    print(year)
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]:
        sex_age = pd.read_csv(
            f"data/Census/Sex by Age Data/ACSDT5Y{year}.B01001-Data.csv",
            header=1,
            low_memory=False,
        )
        sex_age["pct_male_pop_45_64"] = (
            sex_age["Estimate!!Total!!Male!!45 to 49 years"]
            + sex_age["Estimate!!Total!!Male!!50 to 54 years"]
            + sex_age["Estimate!!Total!!Male!!55 to 59 years"]
            + sex_age["Estimate!!Total!!Male!!60 and 61 years"]
            + sex_age["Estimate!!Total!!Male!!62 to 64 years"]
        ) / sex_age["Estimate!!Total!!Male"]

        sex_age["pct_male_pop_0_17"] = (
            sex_age["Estimate!!Total!!Male!!Under 5 years"]
            + sex_age["Estimate!!Total!!Male!!5 to 9 years"]
            + sex_age["Estimate!!Total!!Male!!10 to 14 years"]
            + sex_age["Estimate!!Total!!Male!!15 to 17 years"]
        ) / sex_age["Estimate!!Total!!Male"]

        sex_age["pct_male_pop_18_44"] = (
            sex_age["Estimate!!Total!!Male!!18 and 19 years"]
            + sex_age["Estimate!!Total!!Male!!20 years"]
            + sex_age["Estimate!!Total!!Male!!21 years"]
            + sex_age["Estimate!!Total!!Male!!22 to 24 years"]
            + sex_age["Estimate!!Total!!Male!!25 to 29 years"]
            + sex_age["Estimate!!Total!!Male!!30 to 34 years"]
            + sex_age["Estimate!!Total!!Male!!35 to 39 years"]
            + sex_age["Estimate!!Total!!Male!!40 to 44 years"]
        ) / sex_age["Estimate!!Total!!Male"]

        sex_age["pct_male_pop_65+"] = (
            sex_age["Estimate!!Total!!Male!!65 and 66 years"]
            + sex_age["Estimate!!Total!!Male!!67 to 69 years"]
            + sex_age["Estimate!!Total!!Male!!70 to 74 years"]
            + sex_age["Estimate!!Total!!Male!!75 to 79 years"]
            + sex_age["Estimate!!Total!!Male!!80 to 84 years"]
            + sex_age["Estimate!!Total!!Male!!85 years and over"]
        ) / sex_age["Estimate!!Total!!Male"]

        sex_age["pct_female_pop_0_17"] = (
            sex_age["Estimate!!Total!!Female!!Under 5 years"]
            + sex_age["Estimate!!Total!!Female!!5 to 9 years"]
            + sex_age["Estimate!!Total!!Female!!10 to 14 years"]
            + sex_age["Estimate!!Total!!Female!!15 to 17 years"]
        ) / sex_age["Estimate!!Total!!Female"]

        sex_age["pct_female_pop_18_44"] = (
            sex_age["Estimate!!Total!!Female!!18 and 19 years"]
            + sex_age["Estimate!!Total!!Female!!20 years"]
            + sex_age["Estimate!!Total!!Female!!21 years"]
            + sex_age["Estimate!!Total!!Female!!22 to 24 years"]
            + sex_age["Estimate!!Total!!Female!!25 to 29 years"]
            + sex_age["Estimate!!Total!!Female!!30 to 34 years"]
            + sex_age["Estimate!!Total!!Female!!35 to 39 years"]
            + sex_age["Estimate!!Total!!Female!!40 to 44 years"]
        ) / sex_age["Estimate!!Total!!Female"]

        sex_age["pct_female_pop_45_64"] = (
            sex_age["Estimate!!Total!!Female!!45 to 49 years"]
            + sex_age["Estimate!!Total!!Female!!50 to 54 years"]
            + sex_age["Estimate!!Total!!Female!!55 to 59 years"]
            + sex_age["Estimate!!Total!!Female!!60 and 61 years"]
            + sex_age["Estimate!!Total!!Female!!62 to 64 years"]
        ) / sex_age["Estimate!!Total!!Female"]

        sex_age["pct_female_pop_65+"] = (
            sex_age["Estimate!!Total!!Female!!65 and 66 years"]
            + sex_age["Estimate!!Total!!Female!!67 to 69 years"]
            + sex_age["Estimate!!Total!!Female!!70 to 74 years"]
            + sex_age["Estimate!!Total!!Female!!75 to 79 years"]
            + sex_age["Estimate!!Total!!Female!!80 to 84 years"]
            + sex_age["Estimate!!Total!!Female!!85 years and over"]
        ) / sex_age["Estimate!!Total!!Female"]

        sex_age["pct_female"] = (
            sex_age["Estimate!!Total!!Female"] / sex_age["Estimate!!Total"]
        )

    else:
        sex_age = pd.read_csv(
            f"data/Census/Sex by Age Data/ACSDT5Y{year}.B01001-Data.csv",
            header=1,
            low_memory=False,
        )
        sex_age["pct_male_pop_45_64"] = (
            sex_age["Estimate!!Total:!!Male:!!45 to 49 years"]
            + sex_age["Estimate!!Total:!!Male:!!50 to 54 years"]
            + sex_age["Estimate!!Total:!!Male:!!55 to 59 years"]
            + sex_age["Estimate!!Total:!!Male:!!60 and 61 years"]
            + sex_age["Estimate!!Total:!!Male:!!62 to 64 years"]
        ) / sex_age["Estimate!!Total:!!Male:"]

        sex_age["pct_male_pop_0_17"] = (
            sex_age["Estimate!!Total:!!Male:!!Under 5 years"]
            + sex_age["Estimate!!Total:!!Male:!!5 to 9 years"]
            + sex_age["Estimate!!Total:!!Male:!!10 to 14 years"]
            + sex_age["Estimate!!Total:!!Male:!!15 to 17 years"]
        ) / sex_age["Estimate!!Total:!!Male:"]

        sex_age["pct_male_pop_18_44"] = (
            sex_age["Estimate!!Total:!!Male:!!18 and 19 years"]
            + sex_age["Estimate!!Total:!!Male:!!20 years"]
            + sex_age["Estimate!!Total:!!Male:!!21 years"]
            + sex_age["Estimate!!Total:!!Male:!!22 to 24 years"]
            + sex_age["Estimate!!Total:!!Male:!!25 to 29 years"]
            + sex_age["Estimate!!Total:!!Male:!!30 to 34 years"]
            + sex_age["Estimate!!Total:!!Male:!!35 to 39 years"]
            + sex_age["Estimate!!Total:!!Male:!!40 to 44 years"]
        ) / sex_age["Estimate!!Total:!!Male:"]

        sex_age["pct_male_pop_65+"] = (
            sex_age["Estimate!!Total:!!Male:!!65 and 66 years"]
            + sex_age["Estimate!!Total:!!Male:!!67 to 69 years"]
            + sex_age["Estimate!!Total:!!Male:!!70 to 74 years"]
            + sex_age["Estimate!!Total:!!Male:!!75 to 79 years"]
            + sex_age["Estimate!!Total:!!Male:!!80 to 84 years"]
            + sex_age["Estimate!!Total:!!Male:!!85 years and over"]
        ) / sex_age["Estimate!!Total:!!Male:"]

        sex_age["pct_female_pop_0_17"] = (
            sex_age["Estimate!!Total:!!Female:!!Under 5 years"]
            + sex_age["Estimate!!Total:!!Female:!!5 to 9 years"]
            + sex_age["Estimate!!Total:!!Female:!!10 to 14 years"]
            + sex_age["Estimate!!Total:!!Female:!!15 to 17 years"]
        ) / sex_age["Estimate!!Total:!!Female:"]

        sex_age["pct_female_pop_18_44"] = (
            sex_age["Estimate!!Total:!!Female:!!18 and 19 years"]
            + sex_age["Estimate!!Total:!!Female:!!20 years"]
            + sex_age["Estimate!!Total:!!Female:!!21 years"]
            + sex_age["Estimate!!Total:!!Female:!!22 to 24 years"]
            + sex_age["Estimate!!Total:!!Female:!!25 to 29 years"]
            + sex_age["Estimate!!Total:!!Female:!!30 to 34 years"]
            + sex_age["Estimate!!Total:!!Female:!!35 to 39 years"]
            + sex_age["Estimate!!Total:!!Female:!!40 to 44 years"]
        ) / sex_age["Estimate!!Total:!!Female:"]

        sex_age["pct_female_pop_45_64"] = (
            sex_age["Estimate!!Total:!!Female:!!45 to 49 years"]
            + sex_age["Estimate!!Total:!!Female:!!50 to 54 years"]
            + sex_age["Estimate!!Total:!!Female:!!55 to 59 years"]
            + sex_age["Estimate!!Total:!!Female:!!60 and 61 years"]
            + sex_age["Estimate!!Total:!!Female:!!62 to 64 years"]
        ) / sex_age["Estimate!!Total:!!Female:"]

        sex_age["pct_female_pop_65+"] = (
            sex_age["Estimate!!Total:!!Female:!!65 and 66 years"]
            + sex_age["Estimate!!Total:!!Female:!!67 to 69 years"]
            + sex_age["Estimate!!Total:!!Female:!!70 to 74 years"]
            + sex_age["Estimate!!Total:!!Female:!!75 to 79 years"]
            + sex_age["Estimate!!Total:!!Female:!!80 to 84 years"]
            + sex_age["Estimate!!Total:!!Female:!!85 years and over"]
        ) / sex_age["Estimate!!Total:!!Female:"]

        sex_age["pct_female"] = (
            sex_age["Estimate!!Total:!!Female:"] / sex_age["Estimate!!Total:"]
        )

    sex_age["geo_code"] = sex_age["Geography"].apply(lambda x: str(x)[-5:])
    sex_age["year"] = int(year)

    df = sex_age[
        [
            "geo_code",
            "year",
            "pct_male_pop_0_17",
            "pct_male_pop_18_44",
            "pct_male_pop_45_64",
            "pct_male_pop_65+",
            "pct_female_pop_0_17",
            "pct_female_pop_18_44",
            "pct_female_pop_45_64",
            "pct_female_pop_65+",
            "pct_female",
        ]
    ]
    sex_age_list.append(df)

sex_age_df = pd.concat(sex_age_list, axis=0)


merge_5 = merge_4.merge(sex_age_df, on=["geo_code", "year"], how='left')
merge_5.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323


## Older Adults Living Alone

- Older Adults Living Alone: % of adults 65 years and older who live alone

In [16]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

senior_list = []

for year in year_list:
    print(year)
    senior = pd.read_csv(
        f"data/Census/Senior Data/ACSST5Y{year}.S0103-Data.csv",
        header=1,
        low_memory=False,
    )
    if year in ["2010", "2011", "2012"]:
        senior["older_adults_living_alone"] = senior[
            "65 years and over!!Estimate!!HOUSEHOLDS BY TYPE!!Nonfamily households!!Householder living alone"
        ]
    elif year in ["2013", "2014", "2015", "2016"]:
        senior["older_adults_living_alone"] = senior[
            "65 years and over!!Estimate!!HOUSEHOLDS BY TYPE!!Households!!Nonfamily households!!Householder living alone"
        ]

    else:
        senior["older_adults_living_alone"] = senior[
            "Estimate!!65 years and over!!HOUSEHOLDS BY TYPE!!Households!!Nonfamily households!!Householder living alone"
        ]

    senior["geo_code"] = senior["Geography"].apply(lambda x: str(x)[-5:])
    senior["year"] = int(year)
    df = senior[
        [
            "geo_code",
            "year",
            "older_adults_living_alone",
        ]
    ]
    senior_list.append(df)

senior_df = pd.concat(senior_list, axis=0)

merge_6 = merge_5.merge(senior_df, on=["geo_code", "year"], how='left')
merge_6.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN


## Industry and Occupation

In [17]:
columns = [
    "year",
    "geo_code",
    "civilian_employed_pop_16_plus",
    "management_business_science_arts_occupations",
    "service_occupations",
    "sales_and_office_occupations",
    "natural_resources_construction_maintenance_occupations",
    "production_transportation_material_moving_occupations",
    "industry_civilian_employed_pop_16_plus",
    "agriculture_hunting_and_mining",
    "construction",
    "manufacturing",
    "wholesale_trade",
    "retail_trade",
    "transportation_and_utilities",
    "information",
    "finance_insurance_and_real_estate",
    "professional_management_and_scientific_services",
    "educational_and_health_services",
    "arts_entertainment_and_accommodation_services",
    "other_services",
    "public_administration",
    "employment_status_pop_16_plus",
    "in_labor_force",
    "in_civilian_lf",
    "in_civilian_lf_employed",
    "in_civilian_lf_unemployed",
    "in_armed_forces",
    "not_in_labor_force",
]

ind_occ = census[columns]

ind_occ["pct_agriculture_hunting_and_mining"] = ind_occ[
    "agriculture_hunting_and_mining"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_construction"] = ind_occ["construction"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["pct_manufacturing"] = ind_occ["manufacturing"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["pct_wholesale_trade"] = ind_occ["wholesale_trade"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["pct_retail_trade"] = ind_occ["retail_trade"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["pct_transportation_and_utilities"] = ind_occ[
    "transportation_and_utilities"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_information"] = ind_occ["information"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["pct_finance_insurance_and_real_estate"] = ind_occ[
    "finance_insurance_and_real_estate"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_professional_management_and_scientific_services"] = ind_occ[
    "professional_management_and_scientific_services"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_educational_and_health_services"] = ind_occ[
    "educational_and_health_services"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_arts_entertainment_and_accommodation_services"] = ind_occ[
    "arts_entertainment_and_accommodation_services"
].astype("Int64") / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_public_administration"] = ind_occ["public_administration"].astype(
    "Int64"
) / ind_occ["industry_civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_other_services"] = ind_occ["other_services"].astype("Int64") / ind_occ[
    "industry_civilian_employed_pop_16_plus"
].astype("Int64")
ind_occ["unemployment_rate"] = (
    ind_occ["in_civilian_lf_unemployed"].astype("Int64")
    / ind_occ["in_civilian_lf"].astype("Int64")
) * 100


ind_occ["pct_mgmt_bus_sci_art_occupations"] = ind_occ[
    "management_business_science_arts_occupations"
].astype("Int64") / ind_occ["civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_service_occupations"] = ind_occ["service_occupations"].astype(
    "Int64"
) / ind_occ["civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_sales_and_office_occupations"] = ind_occ[
    "sales_and_office_occupations"
].astype("Int64") / ind_occ["civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_nat_res_constr_maint_occupations"] = ind_occ[
    "natural_resources_construction_maintenance_occupations"
].astype("Int64") / ind_occ["civilian_employed_pop_16_plus"].astype("Int64")
ind_occ["pct_prod_transp_mat_moving_occupations"] = ind_occ[
    "production_transportation_material_moving_occupations"
].astype("Int64") / ind_occ["civilian_employed_pop_16_plus"].astype("Int64")


ind_occ = ind_occ[
    [
        "year",
        "geo_code",
        "pct_agriculture_hunting_and_mining",
        "pct_construction",
        "pct_manufacturing",
        "pct_wholesale_trade",
        "pct_retail_trade",
        "pct_transportation_and_utilities",
        "pct_information",
        "pct_finance_insurance_and_real_estate",
        "pct_professional_management_and_scientific_services",
        "pct_educational_and_health_services",
        "pct_arts_entertainment_and_accommodation_services",
        "pct_public_administration",
        "pct_other_services",
        "unemployment_rate",
        "pct_mgmt_bus_sci_art_occupations",
        "pct_service_occupations",
        "pct_sales_and_office_occupations",
        "pct_nat_res_constr_maint_occupations",
        "pct_prod_transp_mat_moving_occupations",
    ]
]

merge_7 = merge_6.merge(ind_occ, on=['geo_code', 'year'], how='left')
merge_7.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.097676,0.28913,0.201013,0.223973,0.14124,0.144644


## Disconnected Youth

- Disconnected Youth: % of teens and young adults ages 16-19 who are neither working nor in school

In [18]:
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

teen_list = []

for year in year_list:
    print(year)
    teen = pd.read_csv(
        f"data/Census/Teenager Data/ACSST5Y{year}.S0902-Data.csv", header=1
    )
    teen["geo_code"] = teen["Geography"].apply(lambda x: str(x)[-5:])
    if year in ["2010", "2011", "2012"]:
        teen["disconnected_youth"] = (
            teen[
                "Total!!Estimate!!IDLENESS!!Not enrolled in school and not in the labor force"
            ]
            / teen["Total!!Estimate!!Population 15 to 19 years"]
        )
    elif year in ["2013", "2014", "2015", "2016"]:
        teen["disconnected_youth"] = (
            teen[
                "Total!!Estimate!!Population 16 to 19 years!!IDLENESS!!Not enrolled in school and not in the labor force"
            ]
            / teen["Total!!Estimate!!Population 15 to 19 years"]
        )
    else:
        teen["disconnected_youth"] = (
            teen[
                "Estimate!!Total!!Population 16 to 19 years!!IDLENESS!!Not enrolled in school and not in the labor force"
            ]
            / teen["Estimate!!Total!!Population 16 to 19 years"]
        )
    teen["year"] = int(year)

    df = teen[["geo_code", "disconnected_youth", "year"]]
    teen_list.append(df)

teen_df = pd.concat(teen_list, axis=0)


merge_8 = merge_7.merge(teen_df, on=['geo_code', 'year'], how='left')
merge_8.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.097676,0.28913,0.201013,0.223973,0.14124,0.144644,NaN


## Race distribution

- % Non-Hispanic Black
- % American Indian & Alaska Native
- % Asian 
- % Native Hawaiian/Other Pacific Islander 
- % Hispanic
- % Non-Hispanic White

In [22]:
columns = [
    "year",
    "geo_code",
    "hispanic_total_population",
    "hispanic_or_latino",
    "not_hispanic",
    "not_hispanic_white",
    "not_hispanic_black",
    "not_hispanic_indigenous",
    "not_hispanic_asian",
    "not_hispanic_nhpi",
]

race = census[columns]

race["pct_hispanic"] = race["hispanic_or_latino"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")
race["pct_non_hispanic_white"] = race["not_hispanic_white"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")
race["pct_non_hispanic_black"] = race["not_hispanic_black"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")
race["pct_indigenous"] = race["not_hispanic_indigenous"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")
race["pct_asian"] = race["not_hispanic_asian"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")
race["pct_nhpi"] = race["not_hispanic_nhpi"].astype("Int64") / race[
    "hispanic_total_population"
].astype("Int64")

race = race[
    [
        "year",
        "geo_code",
        "pct_hispanic",
        "pct_non_hispanic_white",
        "pct_non_hispanic_black",
        "pct_indigenous",
        "pct_asian",
        "pct_nhpi"
    ]
]

merge_9 = merge_8.merge(race, on=["year", "geo_code"], how='left')
merge_9.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.097676,0.28913,0.201013,0.223973,0.14124,0.144644,NaN,0.014191,0.941647,0.033047,0.001919,0.002349,0.002481


## Housing

- Homeownership: % of occupied housing units that are owned
- Severe Housing Cost Burden: % of household that spends 50% or more of their household income on housing
- Sever housing problems: % of households with at least 1 of 4 housing problems: overcrowding, high housing costs, lack of kitcheb facilities, or lack of plumbing facilities
- Percentage of hosueholds with high housing costs: % of househodls with severe cost burden - monthly housing costs (including utilities) exceed 50% of monthly income
- Percentage of households with overcrowding: % of households with overcrowding - more than 1 person per room
- Percentage of households with lack of kitchen or plumbing faciliteis:

In [23]:
columns = [
    "year",
    "geo_code",
    "housing_tenure_ohu",
    "owner_occupied",
    "renter_occupied",
    "selected_characteristics_ohu",
    "lacking_complete_plumbing_facilities",
    "lacking_complete_kitchen_facilities",
    "occupants_per_room_ohu",
    "1_pt_51_plus_occupants_per_room",
    "grapi_occ_units_paying_rent",
    "grapi_30_34_pt_9_pct",
    "grapi_35_pct_plus",
    "smocapi_huwm",
    "smocapi_huwm_35_pct_plus",
    "smocapi_huwom",
    "smocapi_huwom_35_pct_plus",
]

housing = census[columns]

housing['lacking_complete_kitchen_facilities'] = housing['lacking_complete_kitchen_facilities'].astype(float)
housing['lacking_complete_plumbing_facilities'] = housing['lacking_complete_plumbing_facilities'].astype(float)
housing['1_pt_51_plus_occupants_per_room'] = housing['1_pt_51_plus_occupants_per_room'].astype(float)
housing['grapi_30_34_pt_9_pct'] = housing['grapi_30_34_pt_9_pct'].astype(float)
housing['grapi_35_pct_plus'] = housing['grapi_35_pct_plus'].astype(float)
housing['selected_characteristics_ohu'] = housing['selected_characteristics_ohu'].astype(float)

housing["severe_housing_problems"] = (
    housing["lacking_complete_plumbing_facilities"]
    + housing["lacking_complete_kitchen_facilities"]
    + housing["1_pt_51_plus_occupants_per_room"]
    + housing["grapi_30_34_pt_9_pct"]
    + housing["grapi_35_pct_plus"]
) / housing["selected_characteristics_ohu"]

housing["homeownership"] = housing["owner_occupied"].astype("Int64") / housing[
    "housing_tenure_ohu"
].astype("Int64")

housing["pct_households_with_lack_of_kitchen_or_plumbing_facilities"] = (
    housing["lacking_complete_kitchen_facilities"].astype(float)
    + housing["lacking_complete_plumbing_facilities"]
).astype(float) / housing["selected_characteristics_ohu"].astype(float)
housing["pct_households_with_overcrowding"] = (
    housing["1_pt_51_plus_occupants_per_room"] / housing["selected_characteristics_ohu"]
)

housing["severe_housing_cost_burden"] = (
    housing["grapi_30_34_pt_9_pct"].astype(float)
    + housing["smocapi_huwm_35_pct_plus"].astype(float)
    + housing["smocapi_huwom_35_pct_plus"].astype(float)
) / (
    housing["grapi_occ_units_paying_rent"].astype(float)
    + housing["smocapi_huwm"].astype(float)
    + housing["smocapi_huwom"].astype(float)
)

housing["pct_households_with_high_housing_costs"] = (
    housing["grapi_30_34_pt_9_pct"].astype(float)
    + housing["smocapi_huwm_35_pct_plus"].astype(float)
    + housing["smocapi_huwom_35_pct_plus"].astype(float)
) / (
    housing["grapi_occ_units_paying_rent"].astype(float)
    + housing["smocapi_huwm"].astype(float)
    + housing["smocapi_huwom"].astype(float)
)

housing_df = housing[
    [
        "year",
        "geo_code",
        "severe_housing_problems",
        "homeownership",
        "pct_households_with_lack_of_kitchen_or_plumbing_facilities",
        "pct_households_with_overcrowding",
        "severe_housing_cost_burden",
        "pct_households_with_high_housing_costs",
    ]
]

merge_10 = merge_9.merge(housing_df, on=["year", "geo_code"], how='left')
merge_10.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.097676,0.28913,0.201013,0.223973,0.14124,0.144644,NaN,0.014191,0.941647,0.033047,0.001919,0.002349,0.002481,0.090101,0.777825,0.009916,0.001966,0.132932,0.132932


## Education

- High School Completion: % of adults ages 25 and over with a high school diploma or equivalent
- College Completion:
- Some College: % of adults 25-44 with some post-secondary education

In [24]:
columns = [
    "year",
    "geo_code",
    "pop_25_plus_educ_attainment",
    "high_school_graduate",
    "some_college_no_degree",
    "associates_degree",
    "bachelors_degree",
    "graduate_or_professional_degree",
]

education = census[columns]

education["high_school_completion"] = (
    education["high_school_graduate"].astype("Int64")
    + education["some_college_no_degree"].astype("Int64")
    + education["associates_degree"].astype("Int64")
    + education["bachelors_degree"].astype("Int64")
    + education["graduate_or_professional_degree"].astype("Int64")
) / education["pop_25_plus_educ_attainment"].astype("Int64")


education["college_completion"] = (
    education["bachelors_degree"].astype("Int64")
    + education["graduate_or_professional_degree"].astype("Int64")
) / education["pop_25_plus_educ_attainment"].astype("Int64")

education["some_college"] = (
    education["some_college_no_degree"].astype("Int64")
    + education["associates_degree"].astype("Int64")
    + education["bachelors_degree"].astype("Int64")
    + education["graduate_or_professional_degree"].astype("Int64")
) / education["pop_25_plus_educ_attainment"].astype("Int64")

edu = education[
    ["year", 
     "geo_code", 
     'high_school_completion',
     'college_completion',
     'some_college']
]

merge_11 = merge_10.merge(edu, on=["year", "geo_code"], how="left")
merge_11.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.097676,0.28913,0.201013,0.223973,0.14124,0.144644,NaN,0.014191,0.941647,0

## Children in Single-Parent Households

- Children in Single-Parent Households: % children that live in a household headed by a single parent
- Single Parent Households: % housholds that are single parent

In [25]:
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

hh_list = []

for year in year_list:
    print(year)
    hh = pd.read_csv(
        f"data/Census/Household Data/ACSDT5Y{year}.B09005-Data.csv", header=1
    )
    hh["geo_code"] = hh["Geography"].apply(lambda x: str(x)[-5:])
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]:
        hh["children_in_single_parent_households"] = (
            hh["Estimate!!Total!!In family households!!In male householder, no wife present, family"]
                + hh["Estimate!!Total!!In family households!!In female householder, no husband present, family"]) / hh["Estimate!!Total!!In family households"]
    else:
        hh["children_in_single_parent_households"] = (
            hh["Estimate!!Total:!!In male householder, no spouse/partner present household"] + hh["Estimate!!Total:!!In female householder, no spouse/partner present household"]) / hh["Estimate!!Total:"]
    hh["year"] = int(year)

    df = hh[["geo_code", "children_in_single_parent_households", "year"]]
    hh_list.append(df)

hh_df = pd.concat(hh_list, axis=0)

merge_12 = merge_11.merge(hh_df, on=["geo_code", "year"], how="left")
merge_12.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.01148,0.058428,0.061436,0.214156,0.062465,0.078854,0.044019,7.

In [26]:
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2019",
    "2020",
    "2021",
    "2022",
]

hh_list = []

for year in year_list:
    print(year)
    if year == '2017':
        hh = pd.read_csv(
            f"data/Census/Household Data/ACSST1Y{year}.S1101-Data.csv", header=1
        )
    else:
        hh = pd.read_csv(f"data/Census/Household Data/ACSST5Y{year}.S1101-Data.csv", header=1)

    hh["geo_code"] = hh["Geography"].apply(lambda x: str(x)[-5:])
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]:

        hh['single_parent_households'] = (hh["Male householder, no wife present, family household!!Estimate!!Total households"].astype(int) + hh["Female householder, no husband present, family household!!Estimate!!Total households"].astype(int)) / hh["Total!!Estimate!!Total households"].astype(int)
    else:
        hh['single_parent_households'] = (hh["Estimate!!Male householder, no spouse present, family household!!Total households"].astype(int) + hh["Estimate!!Female householder, no spouse present, family household!!Total households"].astype(int)) / hh["Estimate!!Total!!Total households"].astype(int)

    hh["year"] = int(year)

    df = hh[["geo_code", "single_parent_households", "year"]]
    hh_list.append(df)

hh_df = pd.concat(hh_list, axis=0)

merge_13 = merge_12.merge(hh_df, on=["geo_code", "year"], how="left")
merge_13.head()

2010
2011
2012
2013
2014
2015
2016
2017
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882,0.115164
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.10973,0.024147,0.118993,0.06777,0.0

## Income and Poverty

- Gini Index
- Median Household Income
- Children in Poverty: % of people under age 18 in poverty
- Children Eligible for Free or Reduced Price Lunch: % of children enrolled in public schools that are eligible for free or reduced price lunch
- Poverty: % of population living below the Federal Poverty Line

In [27]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022"
]

gini_list = []

for year in year_list:
    gini = pd.read_csv(
        f"data/Census/Gini Index Data/ACSDT5Y{year}.B19083-Data.csv", header=1
    )
    gini["geo_code"] = gini["Geography"].apply(lambda x: str(x)[-5:])
    gini["gini_index"] = gini["Estimate!!Gini Index"]
    gini["year"] = int(year)

    df = gini[['geo_code', 'gini_index', 'year']]
    gini_list.append(df)

gini_df = pd.concat(gini_list, axis=0)

merge_14 = merge_13.merge(gini_df, on=["geo_code", "year"], how="left")
merge_14.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882,0.115164,0.386
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0.209966,0.475323,NaN,0.071332,0.077191,0.

In [28]:
columns = ["year", "geo_code", "median_hh_income"]

income = census[columns]

merge_15 = merge_14.merge(income, on=["year", "geo_code"], how="left")
merge_15.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882,0.115164,0.386,68253
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171485,0.210138,0.379106,0.274132,0.136624,0.209757,0.303013,0.277264,0

In [29]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

poverty_list = []

for year in year_list:
    print(year)
    if year in ['2010', '2011']:
        poverty = pd.read_csv(f"data/Census/Poverty Data/ACSST1Y{year}.S1701-Data.csv", header=1, low_memory=False)
    else:
        poverty = pd.read_csv(
            f"data/Census/Poverty Data/ACSST5Y{year}.S1701-Data.csv", header=1, low_memory=False
        )
    poverty["geo_code"] = poverty["Geography"].apply(lambda x: str(x)[-5:])
    poverty["year"] = int(year)
    if year in ['2010', '2011', '2012', '2013', '2014', '2015', '2016']:
        poverty["poverty"] = poverty[
            "Percent below poverty level!!Estimate!!Population for whom poverty status is determined"
        ].astype("Float64")
        poverty['poverty_num'] = poverty['Below poverty level!!Estimate!!Population for whom poverty status is determined'].astype('Float64')
        poverty['poverty_den'] = poverty['Total!!Estimate!!Population for whom poverty status is determined'].astype('Float64')
        poverty['poverty'] = poverty['poverty_num'] / poverty['poverty_den']

        poverty['children_poverty_num'] = poverty['Below poverty level!!Estimate!!AGE!!Under 18 years'].astype('Float64')
        poverty['children_poverty_den'] = poverty['Total!!Estimate!!AGE!!Under 18 years'].astype('Float64')
        poverty["children_in_poverty"] = poverty["children_poverty_num"] / poverty["children_poverty_den"]
    else:
        poverty["poverty_num"] = poverty[
            "Estimate!!Below poverty level!!Population for whom poverty status is determined"
        ].astype('Float64')
        poverty["poverty_den"] = poverty[
            "Estimate!!Total!!Population for whom poverty status is determined"
        ].astype("Float64")
        poverty["poverty"] = poverty["poverty_num"] / poverty["poverty_den"]

        poverty["children_poverty_num"] = poverty[
            "Estimate!!Below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years"
        ].astype('Float64')
        poverty["children_poverty_den"] = poverty[
            "Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years"
        ].astype("Float64")
        poverty['children_in_poverty'] = poverty['children_poverty_num'] / poverty['children_poverty_den']

    df = poverty[["geo_code", "year", "poverty", "children_in_poverty"]]
    poverty_list.append(df)

poverty_df = pd.concat(poverty_list, axis=0)

merge_16 = merge_15.merge(poverty_df, on=["geo_code", "year"], how="left")
merge_16.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882,0.115164,0.386,68253,<NA>,<NA>
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN,NaN,NaN,25.462691,0.805110,4.383247,30230,41.4,0.209957,0.342937,0.275620,0.171

In [30]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

public_list = []

for year in year_list:
    print(year)
    df = pd.read_csv(
        f"data/Census/School Data/ACSDT5Y{year}.B14002-Data.csv",
        header=1,
        low_memory=False,
    )

    df["geo_code"] = df["Geography"].apply(lambda x: str(x)[-5:])
    df["year"] = int(year)
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]:

        columns = [
            "geo_code",
            "year",
            "Estimate!!Total!!Male!!Enrolled in school!!Enrolled in nursery school, preschool!!Public school",
            "Estimate!!Total!!Male!!Enrolled in school!!Enrolled in kindergarten!!Public school",
            "Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 1 to grade 4!!Public school",
            "Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 5 to grade 8!!Public school",
            "Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 9 to grade 12!!Public school",
            "Estimate!!Total!!Female!!Enrolled in school!!Enrolled in nursery school, preschool!!Public school",
            "Estimate!!Total!!Female!!Enrolled in school!!Enrolled in kindergarten!!Public school",
            "Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 1 to grade 4!!Public school",
            "Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 5 to grade 8!!Public school",
            "Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 9 to grade 12!!Public school",
            ]
        df = df[columns]

        df['enrolled_in_public'] = df["Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 1 to grade 4!!Public school"].astype('Float64') + df["Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 5 to grade 8!!Public school"].astype('Float64') + df["Estimate!!Total!!Male!!Enrolled in school!!Enrolled in grade 9 to grade 12!!Public school"].astype('Float64') + df["Estimate!!Total!!Male!!Enrolled in school!!Enrolled in kindergarten!!Public school"].astype('Float64') + df["Estimate!!Total!!Male!!Enrolled in school!!Enrolled in nursery school, preschool!!Public school"].astype('Float64') + df["Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 1 to grade 4!!Public school"].astype('Float64') + df["Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 5 to grade 8!!Public school"].astype('Float64') + df["Estimate!!Total!!Female!!Enrolled in school!!Enrolled in grade 9 to grade 12!!Public school"].astype('Float64') + df['Estimate!!Total!!Female!!Enrolled in school!!Enrolled in kindergarten!!Public school'].astype('Float64') + df['Estimate!!Total!!Female!!Enrolled in school!!Enrolled in nursery school, preschool!!Public school'].astype('Float64')

    else:
        columns = [
            "geo_code", 
            "year",
            "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in nursery school, preschool:!!Public school",
            "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in kindergarten:!!Public school",
            "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 1 to grade 4:!!Public school",
            "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 5 to grade 8:!!Public school",
            "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 9 to grade 12:!!Public school",
            "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in nursery school, preschool:!!Public school",
            "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in kindergarten:!!Public school",
            "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 1 to grade 4:!!Public school",
            "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 5 to grade 8:!!Public school",
            "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 9 to grade 12:!!Public school"
            ]
        
        df = df[columns]

        df["enrolled_in_public"] = (
            df[
                "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 1 to grade 4:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 5 to grade 8:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in grade 9 to grade 12:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in kindergarten:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Male:!!Enrolled in school:!!Enrolled in nursery school, preschool:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 1 to grade 4:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 5 to grade 8:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in grade 9 to grade 12:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in kindergarten:!!Public school"
            ].astype("Float64")
            + df[
                "Estimate!!Total:!!Female:!!Enrolled in school:!!Enrolled in nursery school, preschool:!!Public school"
            ].astype("Float64")
        )

    df_2 = pd.read_csv(f"data/Census/School Poverty Data/ACSDT5Y{year}.B14006-Data.csv", header=1)

    df_2["geo_code"] = df_2["Geography"].apply(lambda x: str(x)[-5:])
    df_2["year"] = int(year)
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]:

        columns = ["geo_code", "year", 
            "Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in nursery school, preschool",
            "Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in kindergarten",
            "Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 1 to grade 4",
            "Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 5 to grade 8",
            "Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 9 to grade 12"]

        df_2 = df_2[columns]

        df_2["enrolled_in_public_poverty"] = df_2["Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 1 to grade 4"].astype('Float64') + df_2["Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 5 to grade 8"].astype('Float64') + df_2["Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in grade 9 to grade 12"].astype('Float64') + df_2["Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in kindergarten"].astype('Float64') + df_2["Estimate!!Total!!Income in the past 12 months below the poverty level!!Enrolled in school!!Enrolled in nursery school, preschool"].astype('Float64')

    else:

        columns = ["geo_code", "year", 
            "Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in nursery school, preschool",
            "Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in kindergarten",
            "Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 1 to grade 4",
            "Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 5 to grade 8",
            "Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 9 to grade 12"]

        df_2 = df_2[columns]

        df_2["enrolled_in_public_poverty"] = df_2["Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 1 to grade 4"].astype('Float64') + df_2["Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 5 to grade 8"].astype('Float64') + df_2["Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in grade 9 to grade 12"].astype('Float64') + df_2["Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in kindergarten"].astype('Float64') + df_2["Estimate!!Total:!!Income in the past 12 months below the poverty level:!!Enrolled in school:!!Enrolled in nursery school, preschool"].astype('Float64')


    data = df.merge(df_2, on=["geo_code", "year"], how="left")

    data["children_eligible_for_free_or_reduced_lunch"] = data["enrolled_in_public_poverty"] / data["enrolled_in_public"]

    data = data[["geo_code", "year", "children_eligible_for_free_or_reduced_lunch"]]

    public_list.append(data)

public_df = pd.concat(public_list, axis=0)

merge_17 = merge_16.merge(public_df, on=["geo_code", "year"], how="left")
merge_17.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty,children_eligible_for_free_or_reduced_lunch
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>,0.211972
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>,0.110807
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>,0.159477
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.816641,0.006779,0.000000,0.170185,0.170185,0.910179,0.247213,0.588507,0.187882,0.115164,0.386,68253,<NA>,<NA>,0.039793
4,2010,17135,Illinois,IL,"Montgomery County, Illinois",Montgomery,0.011574,NaN

## Geographic Mobility

In [31]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

mobility_list = []

for year in year_list:
    mobility = pd.read_csv(
            f"data/Census/Geographic Mobility Data/ACSST5Y{year}.S0701-Data.csv",
            header=1,
            low_memory=False,
    )

    mobility["geo_code"] = mobility["Geography"].apply(lambda x: str(x)[-5:])
    mobility["year"] = int(year)
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016"]:
        columns = [
            "geo_code",
            "year",
            "Moved; within same county!!Estimate!!Population 1 year and over",
            "Moved; from different county, same state!!Estimate!!Population 1 year and over",
            "Moved; from different  state!!Estimate!!Population 1 year and over",
            "Moved; from abroad!!Estimate!!Population 1 year and over",
        ]
        df = mobility[columns]
        df.columns = [
            "geo_code",
            "year",
            "pct_moved_within_same_county",
            "pct_moved_from_diff_county_same_state",
            "pct_moved_from_diff_county_diff_state",
            "pct_moved_from_abroad"
        ]
        df["pct_moved_within_same_county"] = (
            df["pct_moved_within_same_county"].astype("Float64") / 100
        )
        df["pct_moved_from_diff_county_same_state"] = (
            df["pct_moved_from_diff_county_same_state"].astype("Float64") / 100
        )
        df["pct_moved_from_diff_county_diff_state"] = (
            df["pct_moved_from_diff_county_diff_state"].astype("Float64") / 100
        )
        df["pct_moved_from_abroad"] = (
            df["pct_moved_from_abroad"].astype("Float64") / 100
        )
    else:
        columns = [
            "geo_code",
            "year",
            "Estimate!!Moved; within same county!!Population 1 year and over",
            "Estimate!!Moved; from different county, same state!!Population 1 year and over",
            "Estimate!!Moved; from different  state!!Population 1 year and over",
            "Estimate!!Moved; from abroad!!Population 1 year and over",
        ]
        df = mobility[columns]
        df.columns = [
            "geo_code",
            "year",
            "pct_moved_within_same_county",
            "pct_moved_from_diff_county_same_state",
            "pct_moved_from_diff_county_diff_state",
            "pct_moved_from_abroad"
        ]
        df["pct_moved_within_same_county"] = (
            df["pct_moved_within_same_county"].astype("Float64") / 100
        )
        df["pct_moved_from_diff_county_same_state"] = (
            df["pct_moved_from_diff_county_same_state"].astype("Float64") / 100
        )
        df["pct_moved_from_diff_county_diff_state"] = (
            df["pct_moved_from_diff_county_diff_state"].astype("Float64") / 100
        )
        df["pct_moved_from_abroad"] = (
            df["pct_moved_from_abroad"].astype("Float64") / 100
        )

    mobility_list.append(df)

mobility_df = pd.concat(mobility_list, axis=0)

merge_18 = merge_17.merge(mobility_df, on=["geo_code", "year"], how="left")
merge_18.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty,children_eligible_for_free_or_reduced_lunch,pct_moved_within_same_county,pct_moved_from_diff_county_same_state,pct_moved_from_diff_county_diff_state,pct_moved_from_abroad
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>,0.211972,0.064,0.013,0.046,0.001
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>,0.110807,0.04,0.064,0.009,0.002
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>,0.159477,0.078,0.023,0.012,0.0
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.001203,0.001264,0.003114,0.000802,0.066016,0.8

In [54]:
merge_16.to_csv('data/census_updated_data_090824.csv')

## Teen Births

- Teen Births: NUmber of births per 1000 female population ages 15-19

In [32]:
# Define a list of years for which data will be downloaded
year_list = [
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
]

birth_list = []

for year in year_list:
    print(year)
    birth = pd.read_csv(
        f"data/Census/birth Data/ACSDT5Y{year}.B13002-Data.csv",
        header=1,
        low_memory=False,
    )
    birth["geo_code"] = birth["Geography"].apply(lambda x: str(x)[-5:])
    birth["year"] = int(year)
    if year in ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]:
        birth["teen_births"] = (
            birth[
                "Estimate!!Total!!Women who had a birth in the past 12 months!!Now married (including separated and spouse absent)!!15 to 19 years old"
            ].astype("Float64")
            + birth[
                "Estimate!!Total!!Women who had a birth in the past 12 months!!Unmarried (never married, widowed, and divorced)!!15 to 19 years old"
            ].astype("Float64")
            ) / (birth[
                "Estimate!!Total!!Women who did not have a birth in the past 12 months!!Now married (including separated and spouse absent)!!15 to 19 years old"
            ].astype("Float64") + birth['Estimate!!Total!!Women who did not have a birth in the past 12 months!!Unmarried (never married, widowed, and divorced)!!15 to 19 years old'].astype("Float64") + birth[
                "Estimate!!Total!!Women who had a birth in the past 12 months!!Now married (including separated and spouse absent)!!15 to 19 years old"
            ].astype("Float64")
            + birth[
                "Estimate!!Total!!Women who had a birth in the past 12 months!!Unmarried (never married, widowed, and divorced)!!15 to 19 years old"
            ].astype("Float64")) * 1000
    else:
        birth["teen_births"] = (
            (
                birth[
                    "Estimate!!Total:!!Women who had a birth in the past 12 months:!!Now married (including separated and spouse absent):!!15 to 19 years old"
                ].astype("Float64")
                + birth[
                    "Estimate!!Total:!!Women who had a birth in the past 12 months:!!Unmarried (never married, widowed, and divorced):!!15 to 19 years old"
                ].astype("Float64")
            )
            / (
                birth[
                    "Estimate!!Total:!!Women who did not have a birth in the past 12 months:!!Now married (including separated and spouse absent):!!15 to 19 years old"
                ].astype("Float64")
                + birth[
                    "Estimate!!Total:!!Women who did not have a birth in the past 12 months:!!Unmarried (never married, widowed, and divorced):!!15 to 19 years old"
                ].astype("Float64")
                + birth[
                    "Estimate!!Total:!!Women who had a birth in the past 12 months:!!Now married (including separated and spouse absent):!!15 to 19 years old"
                ].astype("Float64")
                + birth[
                    "Estimate!!Total:!!Women who had a birth in the past 12 months:!!Unmarried (never married, widowed, and divorced):!!15 to 19 years old"
                ].astype("Float64")
            )
            * 1000
        )
    df = birth[["geo_code", "year", "teen_births"]]
    birth_list.append(df)

birth_df = pd.concat(birth_list, axis=0)

merge_19 = merge_18.merge(birth_df, on=["geo_code", "year"], how="left")
merge_19.head()

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty,children_eligible_for_free_or_reduced_lunch,pct_moved_within_same_county,pct_moved_from_diff_county_same_state,pct_moved_from_diff_county_diff_state,pct_moved_from_abroad,teen_births
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>,0.211972,0.064,0.013,0.046,0.001,14.184397
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>,0.110807,0.04,0.064,0.009,0.002,25.229358
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>,0.159477,0.078,0.023,0.012,0.0,67.307692
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0.124963,0.110261,NaN,0.012981,0.972711,0.0012

## Broadband Access

- Broadband Access: % households with broadband internet connection

## Gender Pay Gap

- Gender Pay Gap:  Ratio of women's median earnings to men's median earnings for all full-time-year-round workers, presented as cents on the dollar

In [33]:
columns = ["year",
           "geo_code",
           'median_earnings_male_ft_yr_workers', 
           'median_earnings_female_ft_yr_workers']

gender_pay = census[columns]

gender_pay['gender_pay_gap'] = gender_pay['median_earnings_female_ft_yr_workers'].astype('Float64') / gender_pay['median_earnings_male_ft_yr_workers'].astype('Float64') * 100

gender_pay = gender_pay[
    ["year", "geo_code", "gender_pay_gap"]
]

merge_20 = merge_19.merge(gender_pay, on=["year", "geo_code"], how="left")
merge_20.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty,children_eligible_for_free_or_reduced_lunch,pct_moved_within_same_county,pct_moved_from_diff_county_same_state,pct_moved_from_diff_county_diff_state,pct_moved_from_abroad,teen_births,gender_pay_gap
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>,0.211972,0.064,0.013,0.046,0.001,14.184397,55.627177
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>,0.110807,0.04,0.064,0.009,0.002,25.229358,86.622807
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>,0.159477,0.078,0.023,0.012,0.0,67.307692,65.874805
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451,4.519258,0.350804,0.15586,0.258113,0

## Census Participation and Voter Turnout

- Census Participation: % of households that self-responded to the 2020 census (by internet, paper questionnaire or telephone)
- Voter Turnout: % of citizen population aged 18 or older who voted in the 2020 US Presidential election

## Geographic Mobility Index

In [34]:
w_within_cnty = 0.25
w_diff_cnty_same_state = 0.5
w_diff_state = 0.75
w_abroad = 1.00

mobility_df["sub_index_wc"] = (
    mobility_df["pct_moved_within_same_county"].astype("Float64") * 1000 * w_within_cnty
)
mobility_df["sub_index_dc_ss"] = (
    mobility_df["pct_moved_from_diff_county_same_state"].astype("Float64")
    * 1000
    * w_diff_cnty_same_state
)
mobility_df["sub_index_dc_ds"] = (
    mobility_df["pct_moved_from_diff_county_diff_state"].astype("Float64")
    * 1000
    * w_diff_state
)
mobility_df["sub_index_abroad"] = (
    mobility_df["pct_moved_from_abroad"].astype("Float64") * 1000 * w_abroad
)

mobility_df["mobility_index"] = (
    mobility_df["sub_index_wc"]
    + mobility_df["sub_index_dc_ss"]
    + mobility_df["sub_index_dc_ds"]
    + mobility_df["sub_index_abroad"]
)

mobility_df["mobility_index_std"] = mobility_df["mobility_index"].apply(
    lambda x: round((x / 363.5 * 100), 2) if type(x) == float else x
)

merge_21 = merge_20.merge(
    mobility_df[["geo_code", "year", "mobility_index_std"]],
    on=["geo_code", "year"],
    how="left",
)
merge_21.head()

,year,geo_code,state,state_code,geo_full_name,geo_name,pct_not_proficient_in_english,uninsured,uninsured_children,uninsured_adults,long_commute_driving_alone,driving_alone_to_work,income_inequality,population,median_age,pct_under_18,pct_age_18_44,pct_age_45_64,pct_65_and_older,pct_male_pop_0_17,pct_male_pop_18_44,pct_male_pop_45_64,pct_male_pop_65+,pct_female_pop_0_17,pct_female_pop_18_44,pct_female_pop_45_64,pct_female_pop_65+,pct_female,older_adults_living_alone,pct_agriculture_hunting_and_mining,pct_construction,pct_manufacturing,pct_wholesale_trade,pct_retail_trade,pct_transportation_and_utilities,pct_information,pct_finance_insurance_and_real_estate,pct_professional_management_and_scientific_services,pct_educational_and_health_services,pct_arts_entertainment_and_accommodation_services,pct_public_administration,pct_other_services,unemployment_rate,pct_mgmt_bus_sci_art_occupations,pct_service_occupations,pct_sales_and_office_occupations,pct_nat_res_constr_maint_occupations,pct_prod_transp_mat_moving_occupations,disconnected_youth,pct_hispanic,pct_non_hispanic_white,pct_non_hispanic_black,pct_indigenous,pct_asian,pct_nhpi,severe_housing_problems,homeownership,pct_households_with_lack_of_kitchen_or_plumbing_facilities,pct_households_with_overcrowding,severe_housing_cost_burden,pct_households_with_high_housing_costs,high_school_completion,college_completion,some_college,children_in_single_parent_households,single_parent_households,gini_index,median_hh_income,poverty,children_in_poverty,children_eligible_for_free_or_reduced_lunch,pct_moved_within_same_county,pct_moved_from_diff_county_same_state,pct_moved_from_diff_county_diff_state,pct_moved_from_abroad,teen_births,gender_pay_gap,mobility_index_std
0,2010,17127,Illinois,IL,"Massac County, Illinois",Massac,0.002624,NaN,NaN,NaN,15.624765,0.865234,4.229339,15425,41.8,0.230146,0.311313,0.276499,0.182042,0.237822,0.315818,0.286946,0.159414,0.223235,0.307256,0.267094,0.202415,0.526224,NaN,0.018199,0.059068,0.095312,0.018507,0.117212,0.120913,0.009099,0.038402,0.068785,0.213603,0.137107,0.057372,0.046422,6.110628,0.235349,0.225632,0.25987,0.121993,0.157156,NaN,0.018541,0.902626,0.043177,0.002075,0.00363,0.0,0.070713,0.802479,0.007469,0.003814,0.153291,0.153291,0.849575,0.141457,0.495562,0.222385,0.087717,0.396,41077,<NA>,<NA>,0.211972,0.064,0.013,0.046,0.001,14.184397,55.627177,15.96
1,2010,17129,Illinois,IL,"Menard County, Illinois",Menard,0.001754,NaN,NaN,NaN,40.129367,0.788037,3.822340,12702,42.3,0.236026,0.308849,0.303102,0.152023,0.248227,0.311251,0.303836,0.136686,0.224377,0.306556,0.302401,0.166667,0.511573,NaN,0.044374,0.060124,0.055581,0.031955,0.089505,0.058761,0.020445,0.08905,0.067545,0.222778,0.076177,0.128729,0.054975,3.507234,0.348932,0.165834,0.268211,0.0995,0.117522,NaN,0.008109,0.990159,0.000236,0.000236,0.0,0.0,0.069027,0.825158,0.009098,0.000000,0.102008,0.102008,0.903002,0.241528,0.521947,0.232488,0.110759,0.377,56230,<NA>,<NA>,0.110807,0.04,0.064,0.009,0.002,25.229358,86.622807,13.96
2,2010,17131,Illinois,IL,"Mercer County, Illinois",Mercer,0.005386,NaN,NaN,NaN,39.743189,0.809647,3.526119,16551,43.0,0.229412,0.299801,0.297565,0.173222,0.236003,0.302449,0.306632,0.154916,0.223053,0.297246,0.288818,0.190883,0.508972,NaN,0.054232,0.097211,0.157907,0.025141,0.097091,0.062373,0.017,0.049204,0.056267,0.219322,0.056866,0.061415,0.045972,6.177693,0.287322,0.158386,0.227822,0.141626,0.184844,NaN,0.015407,0.970999,0.003625,0.002115,0.002115,0.000785,0.059312,0.799475,0.002769,0.000000,0.136908,0.136908,0.887152,0.145053,0.479537,0.263701,0.149228,0.383,50909,<NA>,<NA>,0.159477,0.078,0.023,0.012,0.0,67.307692,65.874805,11.00
3,2010,17133,Illinois,IL,"Monroe County, Illinois",Monroe,0.008292,NaN,NaN,NaN,45.224157,0.816812,3.728533,32431,40.1,0.248312,0.328266,0.285098,0.138324,0.261490,0.329841,0.292173,0.116495,0.235423,0.326726,0.278178,0.159673,0.505566,NaN,0.019064,0.095858,0.12054,0.035977,0.087731,0.065499,0.033825,0.080739,0.089643,0.205582,0.079723,0.038367,0.047451

In [36]:
merge_21.to_csv('data/census_revised_data_091024.csv')